In [63]:
from multiprocessing import Pool
import pickle
import gzip
import numpy as np
import os, os.path
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from PIL import Image, ImageOps
from itertools import repeat
from sklearn.model_selection import train_test_split
import re
import keras
from keras import backend as K
from data_manager import DataManager, load_img
from data_generator import CustomDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from datetime import datetime
from tensorflow.keras.layers import (
    Input,
    concatenate,
    Convolution2D,
    Flatten,
    BatchNormalization,
    Dropout,
    Conv2D,
    UpSampling2D,
    ELU,
    Dense
)
import tensorflow
import tensorflow.keras.activations as activations
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [45]:
BASE = os.getcwd()
manager = DataManager()
BATCH_SIZE = 32
EPOCHS = 200

In [53]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

def resize_with_padding(img, expected_size):
    """
    this function only works when scaling UP i.e. left,right,top,bottom > 0
    """
    desired_size = expected_size
    height, width = img.shape[:2]
    delta_width = desired_size[1] - width
    delta_height = desired_size[0] - height
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    left, top, right, bottom = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    img = cv2.resize(img, expected_size)
    color = [0,0,0]
    new_img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT,
        value=color)
    return new_img

def grays_to_RGB(img):
    # turn image into grayscale RGB
    return np.array(Image.fromarray(img).convert("RGB"))

def save_img(img, img_idx, path, pid, is_mask=False):
    filename = path + '/' + str(pid) + '_' + str(img_idx) 
    if is_mask: 
        filename += '_mask.png' 
        img = np.asarray(img, dtype="uint8") # convert bool mask into uint8 so cv2 doesn't scream
    else:
        filename += '.png'
        img = grays_to_RGB(img)
    
    cv2.imwrite(filename, img)

def make_dir(path):
    try:
        os.mkdir(path)
    except OSError:
        print (f"Creation of the directory {path} failed", end='\r')

def gen_dataset(imgs, dataset, pid, labels=None, typeof_dataset=None):
    output_dir = BASE + '/data/'+dataset+'/'
    if os.path.isdir(output_dir) is False:
        make_dir(output_dir)
    if typeof_dataset is not None: # this is only for train
        output_dir+=typeof_dataset + '/'
        if os.path.isdir(output_dir) is False:
            make_dir(output_dir)
    
    for i, img in enumerate(imgs):
        save_img(img, i, output_dir, pid)
        if labels is not None: # this is only for train
            save_img(labels[i], i, output_dir, pid, is_mask=True)
    
def list_images(directory, ext='jpg|jpeg|bmp|png|tif'):
    return [os.path.join(directory, f) for f in os.listdir(directory)
            if os.path.isfile(os.path.join(directory, f)) and re.match('([\w]+\.(?:' + ext + '))', f)]

def preprocess(img, denoise=False):
    """
    Preprocess step after image augmentation, and before feeding into conv net.
    """
    if denoise:
        img = cv2.fastNlMeansDenoising(img, h=7)
    img = cv2.resize(img, (DataManager.EX_IMG_TARGET_COLS, DataManager.EX_IMG_TARGET_COLS))
    return img


def transform(img, mask, augment=True):
    """
    Transforms an (img, mask) pair with same augmentation params
    """
    if augment:
        pass
        #img, mask = augmenter.augment_batch(np.array([img, mask]), same_transform=True)
    img = preprocess(img)
    mask = preprocess(mask).astype('float32')
    return np.array([img]), np.array([mask])

In [5]:
train_data = load_zipped_pickle("data/train.pkl")
test_data = load_zipped_pickle("data/test.pkl")

In [6]:
for data in tqdm(train_data, total=len(train_data)):
    imgs = data['video'].T
    typeof_ds = data['dataset']
    labels = data['label'].T
    pacient = data['name']
    gen_dataset(imgs, "train", pacient, labels, typeof_ds)

for data in tqdm(test_data, total=len(test_data)):
    imgs = data['video'].T
    pacient = data['name']
    gen_dataset(imgs, "test", pacient)

100%|██████████| 65/65 [01:08<00:00,  1.05s/it]


In [7]:
len(list_images(BASE+'/data/train/expert'))

3268

In [8]:
len(list_images(BASE+'/data/train/amateur'))

16340

In [10]:
min_w = 1000
max_w = 0
min_h = 1000
max_h = 0
for data in tqdm(test_data, total=len(test_data)):
    imgs = data['video'].T
    pacient = data['name']
    if min_w > imgs.shape[1]:
        min_w = imgs.shape[1]
    if max_w < imgs.shape[1]:
        max_w = imgs.shape[1]
    if min_h > imgs.shape[2]:
        min_h = imgs.shape[2]
    if max_h < imgs.shape[2]:
        max_h = imgs.shape[2]

print("Min width: {} Max width: {}".format(min_w, max_w))
print("Min height: {} Max height: {}".format(min_h, max_h))

100%|██████████| 20/20 [00:00<00:00, 180788.97it/s]

Min width: 600 Max width: 1007
Min height: 582 Max height: 732


It seems scaling down for now should be the way to go:
https://datascience.stackexchange.com/questions/30396/why-do-we-scale-down-images-before-feeding-them-to-the-network
Let's try 224 x 224

In [12]:
amateur, expert = manager.read_train_images() # just for demonstration purposes, this gets called internally

Loading training amateur images...
Done: 0/8170 images
Done: 100/8170 images
Done: 200/8170 images
Done: 300/8170 images
Done: 400/8170 images
Done: 500/8170 images
Done: 600/8170 images
Done: 700/8170 images
Done: 800/8170 images
Done: 900/8170 images
Done: 1000/8170 images
Done: 1100/8170 images
Done: 1200/8170 images
Done: 1300/8170 images
Done: 1400/8170 images
Done: 1500/8170 images
Done: 1600/8170 images
Done: 1700/8170 images
Done: 1800/8170 images
Done: 1900/8170 images
Done: 2000/8170 images
Done: 2100/8170 images
Done: 2200/8170 images
Done: 2300/8170 images
Done: 2400/8170 images
Done: 2500/8170 images
Done: 2600/8170 images
Done: 2700/8170 images
Done: 2800/8170 images
Done: 2900/8170 images
Done: 3000/8170 images
Done: 3100/8170 images
Done: 3200/8170 images
Done: 3300/8170 images
Done: 3400/8170 images
Done: 3500/8170 images
Done: 3600/8170 images
Done: 3700/8170 images
Done: 3800/8170 images
Done: 3900/8170 images
Done: 4000/8170 images
Done: 4100/8170 images
Done: 4200/

In [13]:
amateur[1].shape

(8170, 112, 112)

In [14]:
expert[1].shape

(1634, 224, 224)

In [15]:
manager.create_train_data()

Loading training amateur images...
Done: 0/8170 images
Done: 100/8170 images
Done: 200/8170 images
Done: 300/8170 images
Done: 400/8170 images
Done: 500/8170 images
Done: 600/8170 images
Done: 700/8170 images
Done: 800/8170 images
Done: 900/8170 images
Done: 1000/8170 images
Done: 1100/8170 images
Done: 1200/8170 images
Done: 1300/8170 images
Done: 1400/8170 images
Done: 1500/8170 images
Done: 1600/8170 images
Done: 1700/8170 images
Done: 1800/8170 images
Done: 1900/8170 images
Done: 2000/8170 images
Done: 2100/8170 images
Done: 2200/8170 images
Done: 2300/8170 images
Done: 2400/8170 images
Done: 2500/8170 images
Done: 2600/8170 images
Done: 2700/8170 images
Done: 2800/8170 images
Done: 2900/8170 images
Done: 3000/8170 images
Done: 3100/8170 images
Done: 3200/8170 images
Done: 3300/8170 images
Done: 3400/8170 images
Done: 3500/8170 images
Done: 3600/8170 images
Done: 3700/8170 images
Done: 3800/8170 images
Done: 3900/8170 images
Done: 4000/8170 images
Done: 4100/8170 images
Done: 4200/

In [16]:
manager.create_test_data()

Creating test images...
Done: 0/1572 images
Done: 100/1572 images
Done: 200/1572 images
Done: 300/1572 images
Done: 400/1572 images
Done: 500/1572 images
Done: 600/1572 images
Done: 700/1572 images
Done: 800/1572 images
Done: 900/1572 images
Done: 1000/1572 images
Done: 1100/1572 images
Done: 1200/1572 images
Done: 1300/1572 images
Done: 1400/1572 images
Done: 1500/1572 images
Saving test samples...
Saving to .npy files done.


In [4]:
X_train, X_val, y_train, y_val = manager.load_train_val_data("expert")

In [5]:
y_train.shape

(1470, 224, 224)

In [7]:
X_val.shape

(164, 224, 224)

In [43]:
smooth = 1

def dice(y_true, y_pred):
    """
    Average dice across all samples
    """
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_loss(y_true, y_pred):
    return -dice(y_true, y_pred)

# Helper to build a conv -> BN -> relu block
def _conv_bn_relu(filters, k_row, k_col, strides=(1, 1)):
    def f(input):
        conv = Convolution2D(filters=filters, kernel_size=[k_row, k_col],
                             strides=strides, kernel_initializer="he_normal",
                             padding="same")(input)
        norm = BatchNormalization()(conv)
        return ELU()(norm)
    return f


def build_model(optimizer=None):
    if optimizer is None:
        optimizer = Adam(learning_rate=1e-4)

    inputs = Input((1, DataManager.EX_IMG_TARGET_ROWS, DataManager.EX_IMG_TARGET_COLS), name='main_input')
    conv1 = _conv_bn_relu(32, 7, 7)(inputs)
    conv1 = _conv_bn_relu(32, 3, 3)(conv1)
    pool1 = _conv_bn_relu(32, 2, 2, strides=(2, 2))(conv1)
    drop1 = Dropout(0.5)(pool1)

    conv2 = _conv_bn_relu(64, 3, 3)(drop1)
    conv2 = _conv_bn_relu(64, 3, 3)(conv2)
    pool2 = _conv_bn_relu(64, 2, 2, strides=(2, 2))(conv2)
    drop2 = Dropout(0.5)(pool2)

    conv3 = _conv_bn_relu(128, 3, 3)(drop2)
    conv3 = _conv_bn_relu(128, 3, 3)(conv3)
    pool3 = _conv_bn_relu(128, 2, 2, strides=(2, 2))(conv3)
    drop3 = Dropout(0.5)(pool3)

    conv4 = _conv_bn_relu(256, 3, 3)(drop3)
    conv4 = _conv_bn_relu(256, 3, 3)(conv4)
    pool4 = _conv_bn_relu(256, 2, 2, strides=(2, 2))(conv4)
    drop4 = Dropout(0.5)(pool4)

    conv5 = _conv_bn_relu(512, 3, 3)(drop4)
    conv5 = _conv_bn_relu(512, 3, 3)(conv5)

    drop5 = Dropout(0.5)(conv5)
    # Using conv to mimic fully connected layer.
    aux = Dense(512, kernel_initializer="he_normal", activation='sigmoid')(drop5)
    #Convolution2D(filters=1, kernel_size=[drop5.shape[1], drop5.shape[2]],
    #                    strides=(1, 1), kernel_initializer="he_normal", activation='sigmoid')(drop5)
    
    aux = Flatten(name='aux_output')(aux)

    up6 = concatenate([UpSampling2D(size=(1,2))(drop5), conv4], axis=-1)
    conv6 = _conv_bn_relu(256, 3, 3)(up6)
    conv6 = _conv_bn_relu(256, 3, 3)(conv6)
    drop6 = Dropout(0.5)(conv6)

    up7 = concatenate([UpSampling2D(size=(1,2))(drop6), conv3], axis=3)
    conv7 = _conv_bn_relu(128, 3, 3)(up7)
    conv7 = _conv_bn_relu(128, 3, 3)(conv7)
    drop7 = Dropout(0.5)(conv7)

    up8 = concatenate([UpSampling2D(size=(1,2))(drop7), conv2], axis=3)
    conv8 = _conv_bn_relu(64, 3, 3)(up8)
    conv8 = _conv_bn_relu(64, 3, 3)(conv8)
    drop8 = Dropout(0.5)(conv8)

    up9 = concatenate([UpSampling2D(size=(1,2))(drop8), conv1],axis=3)
    conv9 = _conv_bn_relu(32, 3, 3)(up9)
    conv9 = _conv_bn_relu(32, 3, 3)(conv9)
    drop9 = Dropout(0.5)(conv9)

    conv10 = Convolution2D(1, 1, 1, activation='sigmoid', kernel_initializer="he_normal", name='main_output')(drop9)

    model = Model(inputs=inputs, outputs=[conv10, aux])
    model.compile(optimizer=optimizer,
                  loss={'main_output': dice_loss, 'aux_output': 'binary_crossentropy'},
                  metrics={'main_output': dice, 'aux_output': 'acc'},
                  loss_weights={'main_output': 1, 'aux_output': 0.5})

    return model

In [58]:
model = build_model()

In [59]:
train_generator = CustomDataGenerator(X_train, y_train, transform, BATCH_SIZE)

# Use fixed samples instead to visualize histograms. There is currently a bug that prevents it
# when a val generator is used.
# Not aug val samples to keep the eval consistent.
val_generator = CustomDataGenerator(X_val, y_val, lambda x, y: transform(x, y, augment=False), BATCH_SIZE)

In [60]:
run_id = str(datetime.now())
model_checkpoint = ModelCheckpoint('./results/net.hdf5', monitor='val_loss', save_best_only=True)
tb = TensorBoard(log_dir='./logs/{}'.format(run_id), histogram_freq=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=4, min_lr=1e-6)
print('Training on model')
model.summary()

Training on model
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 1, 224, 224) 0                                            
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 1, 224, 32)   351264      main_input[0][0]                 
__________________________________________________________________________________________________
batch_normalization_22 (BatchNo (None, 1, 224, 32)   128         conv2d_22[0][0]                  
__________________________________________________________________________________________________
elu_22 (ELU)                    (None, 1, 224, 32)   0           batch_normalization_22[0][0]     
__________________________________________________________________________

In [62]:
# from skimage import transform as tf
# import numpy as np
# import random

# augmenter = ImageAugmenter(DataManager.EX_IMG_TARGET_COLS, DataManager.EX_IMG_TARGET_ROWS,
#                            hflip=False, vflip=False,
#                            rotation_deg=5,
#                            translation_x_px=10,
#                            translation_y_px=10)
# new_x, new_y = transform(X_train[100], y_train[100])

In [61]:
model.fit_generator(train_generator, validation_data=val_generator, validation_steps=X_val.shape[0],
                     steps_per_epoch=X_train.shape[0], epochs=EPOCHS, verbose=2,
                     callbacks=[model_checkpoint, reduce_lr, tb], max_queue_size=1000)

Epoch 1/200


NotFoundError:  No algorithm worked!
	 [[node model_1/conv2d_22/Conv2D (defined at <ipython-input-61-4c043c714309>:3) ]] [Op:__inference_train_function_8244]

Function call stack:
train_function
